# Build a RAG Agent for Document Q&As using Strands Agents and Chroma

This notebook demonstrates how to build a Retrieval-Augmented Generation (RAG) agent from scratch using the Strands Agent framework and the Chroma open-source vector database. By the end of this tutorial, you will have a fully functional RAG agent that can answer questions based on a set of provided documents.

## 1. Introduction

### What is RAG?

Retrieval-Augmented Generation (RAG) is a technique that combines the power of large language models (LLMs) with external knowledge sources. Instead of relying solely on the information it was trained on, a RAG system retrieves relevant information from a knowledge base and uses that information to generate a more accurate and context-aware response. A RAG agent will have the following components:

- **Document parser**: Loads documents from a source to markdown (e.g., a text file or PDF).
- **Document chunking**: Splits the loaded documents into smaller chunks.
- **Embedding Model**: Converts the text chunks into numerical vectors (embeddings).
- **Vector Database**: Stores the embeddings and allows for efficient similarity search.
- **Retriever**: Queries the vector database to find the most relevant document chunks for a given question.
- **LLM Agent**: Takes a user's question, uses the retriever tool to get relevant context, and then generates an answer using an LLM.

![RAG_illustration](https://miro.medium.com/v2/resize:fit:1400/format:webp/0*kHb0TZ0IlPIpB-jg.png)

## 2. Setup and Installation

First, let's install the necessary packages. We'll need `strands-agents` for our agent, `chromadb` for our vector store, `docling` to parse documents into markdown format and `chonkie` to recursively chunk documents.

In [ ]:
!pip install strands-agents chromadb docling chonkie

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.8/345.8 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.0/304.0 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 114.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.2/385.2 kB 43.6 MB/s eta 0

## 3. Setup Chroma vector database

Guide to choose the right vector database:
![Core concepts of a Strands agent](https://miro.medium.com/v2/resize:fit:1400/format:webp/1*lvEKdegbcJm_uChazzEPxg.png)

[Chroma](https://www.trychroma.com/) is an open-source embedding database that makes it easy to build AI applications with semantic search capabilities. We will use Chroma to store our document embeddings and retrieve them based on semantic similarity.

*   Simple: Fully-typed, fully-tested, fully-documented == happiness

*   Integrations: 🦜️🔗 LangChain (python and js), 🦙 LlamaIndex and more soon
*   Dev, Test, Prod: the same API that runs in your python notebook, scales to your cluster
*   Feature-rich: Queries, filtering, regex and more
Free & Open Source: Apache 2.0 Licensed


![Core concepts of a Strands agent](https://miro.medium.com/0*jmcTxk-vcU9d1Rla.png)

Now, we will load the embedding models, process the documents into vectors and store them in our Chroma vector database. We will use a state-of-the-art open-source embedding model: **Qwen3-Embedding-0.6B** given the limited memory on colab. This model, developed by Alibaba Cloud, is known for its excellent performance on the MTEB leaderboard and its strong multilingual capabilities.


**Key Features of Qwen3-Embedding-0.6B:**
- **Retrieval Score**: 64.65 (among the top 10 for embedding models)
- **Context Length**: 32,768 tokens
- **Embedding Dimension**: Up to 2560 (configurable)
- **Multilingual**: Supports 100+ languages
- **Instruction-Aware**: Can use custom instructions for better task-specific performance

![Qwen-embedding-performance](https://i.ibb.co/cS7yXp68/Screenshot-2026-01-20-at-4-18-16-PM.png)

### 3.1. Create Chroma database with the Qwen3-Embedding-0.6B Embedding model

Now, we will load the `Qwen/Qwen3-Embedding-0.6B` model using the `embedding_functions`.

In [ ]:
import chromadb
from chromadb.utils import embedding_functions

doc_chunks = [
    "Artificial intelligence has a long history, dating back to the mid-20th century. Early pioneers like Alan Turing and John McCarthy laid the groundwork for the field. The development of machine learning in recent decades has led to rapid advancements.",
    "Large language models (LLMs) are a type of AI model trained on massive amounts of text data. They can generate human-like text, translate languages, and answer questions in an informative way. GPT-3, developed by OpenAI, is a well-known example.",
    "Vector databases are essential for building modern AI applications. They store and query high-dimensional vectors, such as those generated by embedding models. This enables semantic search and other advanced features in applications like RAG agents."
]

# Load the Qwen3-Embedding-0.6B model
embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="Qwen/Qwen3-Embedding-0.6B",
    device="cuda"
)

# 2. Pass this function when retrieving your collection
client = chromadb.PersistentClient(path="./rag_vec_db")
collection = client.get_or_create_collection(
    name="rag_collection",
    embedding_function=embedding_func,
    metadata={"hnsw:space": "cosine"} # Change the similarity here to cosine distance
)

collection.add(
    documents=doc_chunks,
    ids=[f"id_{i}" for i in range(len(doc_chunks))]
)

print(f"Successfully Create chroma vector database!")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

Successfully Create chroma vector database!


### 3.2. Querying the Vector Database

In [ ]:
top_n = 2
results = collection.query(
    query_texts=["What is vector database?"],
    include=["documents", "distances", "embeddings"],
    n_results=top_n
) # 'ids', 'documents', 'metadatas', and 'embeddings' are returned


print("\n".join(results['documents'][0]))
print("Distance: ", results['distances'][0])
print("Embedding dim: ", len(results['embeddings'][0][0]))

Vector databases are essential for building modern AI applications. They store and query high-dimensional vectors, such as those generated by embedding models. This enables semantic search and other advanced features in applications like RAG agents.
Large language models (LLMs) are a type of AI model trained on massive amounts of text data. They can generate human-like text, translate languages, and answer questions in an informative way. GPT-3, developed by OpenAI, is a well-known example.
Distance:  [0.23217558860778809, 0.6043475866317749]
Embedding dim:  1024


## 4. Creat a RAG agent

### 4.1. Create a RAG tool for Strands Agent

We will create a new RAG tool that queries our Qwen3-powered vector database. To define the tool for Strands agent, the function needs to be defined by @tool.

[Strands Agents](https://strandsagents.com/) is an open-source SDK that simplifies the development of AI agents. It provides a model-driven approach to building and running agents, allowing you to focus on the agent's logic and tools rather than the underlying infrastructure.

![Core concepts of a Strands agent](https://d2908q01vomqb2.cloudfront.net/ca3512f4dfa95a03169c5a670a4c91a19b3077b4/2025/05/16/agentic-loop.png)

In [ ]:
from strands import tool

@tool
def rag_tool(query: str) -> str:
    """Retrieve the relevant document according to the query from vector database."""
    # Query the Qwen3 collection to find the most relevant document chunks
    results = collection.query(
        query_texts=[query],
        n_results=2,
    )

    # Combine the retrieved chunks into a single context
    retrieved_context = "\n\n".join(results['documents'][0])

    return retrieved_context

print("RAG tool created successfully.")

RAG tool created successfully.


### 4.2. Create a Strands Agent with the RAG tool

Finally, we will create a new Strands agent that uses our Qwen-powered RAG tool.

Pre-requisite:
1. Setup AWS account (free credit $100): [register](https://signin.aws.amazon.com/signup?request_type=register)
2. Go to IAM (Identity and Access Management) to create a new user
3. Attach Policy "AmazonBedrockFullAccess" to the user
4. Go to created user and Click "Create access key"

Video tutorial: [Connect with Bedrock backend](https://youtu.be/-Wp3WdyzvXg?t=1078)

**Alternative option**
Setup a local ollama server running model locally:
[Build your first FREE AI Agent](https://colab.research.google.com/drive/12Djry5TI_NPxlzH0hnITT7lPfe-K0_JX?usp=sharing)

In [ ]:
from strands import Agent
from strands.models.bedrock import BedrockModel
import os
from google.colab import userdata

# Load AWS credentials from Colab secrets
os.environ["AWS_ACCESS_KEY_ID"] = userdata.get("AWS_ACCESS_KEY_ID")
os.environ["AWS_SECRET_ACCESS_KEY"] = userdata.get("AWS_SECRET_ACCESS_KEY")
os.environ["AWS_DEFAULT_REGION"] = userdata.get("AWS_DEFAULT_REGION")

from strands import Agent

# Use the US inference profile ID
model = BedrockModel(
    model_id="us.anthropic.claude-sonnet-4-20250514-v1:0"  # Note the "us." prefix
)

agent = Agent(model=model, callback_handler=None,
              tools=[rag_tool],
              system_prompt="""You are a helpful assistant that uses the RAG tool to answer
              questions based on the provided documents. Always cite the retrieved
              information in your responses.""")

print("Chroma-powered RAG agent created successfully.")

Chroma-powered RAG agent created successfully.


### 4.3. Invoke the Agent

Let's test our new agent to see how the Qwen3 embeddings improve the retrieval and response quality.

In [ ]:
query1 = "Is AI helping skill formation or not?"
print(f"Query: {query1}")
response1 = str(agent(query1))
print(f"Agent Response: {response1}")


Query: Is AI helping skill formation or not?
Agent Response: Based on the retrieved documents, I can provide you with a comprehensive answer about AI's impact on skill formation:

**AI has a mixed impact on skill formation, with largely negative effects but some potential for positive outcomes depending on how it's used.**

## Key Findings:

**Negative Impact:**
- The research found that **using AI to complete tasks that require a new skill reduces skill formation**
- In a randomized controlled trial, participants using AI assistance showed **erosion of conceptual understanding, code reading, and debugging skills** compared to those using only web search and instructions
- This suggests that workers acquiring new skills should be **mindful of their reliance on AI during the learning process**

**Critical Factor - How AI is Used:**
The study revealed a **stark divide in outcomes** based on how participants interacted with AI:

- **High-scoring patterns (65%-86% quiz scores):** Participa

**Congratulations!** You have successfully built a RAG agent using the Strands Agent framework and Chroma.

This notebook provides a basic foundation. You can extend this by:

- Using more complex document loaders for different file types.
- Implementing more sophisticated text splitting and chunking strategies.
- Using a more powerful embedding model.
- Adding more tools to your agent.

## 5. (Advanced) Chunking a long document

### Upload PDF
Provide code for the user to upload a PDF file from their local machine to the Colab environment using `google.colab.files`.


In [ ]:
from google.colab import files

print("Please upload your PDF file (e.g., your_book.pdf):")
uploaded = files.upload()

input_doc = list(uploaded.keys())[0]
print(input_doc)

Please upload your PDF file (e.g., your_book.pdf):


Saving AI_Skill_formation.pdf to AI_Skill_formation (1).pdf
AI_Skill_formation (1).pdf


### Parse and Chunk document
Here we use `docling` to convert the pdf into a markdown format that best perserves the sections and layout of documents. And then we use `chonkie` to chunk the documents with custom delimiters. You can also try using the `HybridChunker` directly provided by docling:

`chunker = HybridChunker()`

`doc_chunks = chunker.chunk(result.document)`

In [ ]:
from typing import Text
from chonkie import RecursiveChunker, RecursiveRules, RecursiveLevel
from docling.document_converter import DocumentConverter

def chunk_pdf_by_chapters(pdf_path: str, chunk_size = 2048) -> list:
    """
    Chunks a PDF by chapters and sections using a Markdown-based approach and Chonkie.

    Args:
        pdf_path: URL or The file path to the PDF document.

    Returns:
        A list of doc chunks.
    """

    # 1. Convert PDF to Markdown using docling (preserves structure)
    try:
        converter = DocumentConverter()
        result = converter.convert(pdf_path)
        pdf_markdown = result.document.export_to_markdown()
    except Exception as e:
        print(f"Error converting PDF to Markdown: {e}")
        return []

    # 2. Initialize Chonkie for advanced text splitting
    custom_rules = RecursiveRules(
      levels=[
          # Level 1: Split by markdown-style headers
          RecursiveLevel(delimiters=["#", "##", "###", "####", "#####"], include_delim="next"),

          # Level 2: Split by paragraph breaks (double newlines)
          RecursiveLevel(delimiters=["\n\n"]),

          # Level 3: Split by standard sentence punctuation
          RecursiveLevel(delimiters=[". ", "! ", "? "], include_delim="prev"),

          # Level 4: Fallback to split at whitespace if chunk is still too large
          RecursiveLevel(whitespace=True)])

    chunker = RecursiveChunker(
        rules=custom_rules,
        chunk_size=chunk_size
    )

    # 3. Chunk the markdown text using Chonkie
    raw_chunks = chunker.chunk(pdf_markdown)

    # 4. Convert raw string chunks into LangChain Document objects
    chunks = [chunk.text for chunk in raw_chunks]

    return chunks

In [ ]:
chunks = chunk_pdf_by_chapters(input_doc, 2048)
print("number of chunks", len(chunks))
print(chunks[0])

[INFO] 2026-02-06 03:11:34,893 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2026-02-06 03:11:34,911 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.onnx
[INFO] 2026-02-06 03:11:34,912 [RapidOCR] main.py:53: Using /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.onnx
[INFO] 2026-02-06 03:11:35,027 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2026-02-06 03:11:35,030 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_ppocr_mobile_v2.0_cls_infer.onnx
[INFO] 2026-02-06 03:11:35,031 [RapidOCR] main.py:53: Using /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_ppocr_mobile_v2.0_cls_infer.onnx
[INFO] 2026-02-06 03:11:35,087 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2026-02-06 03:11:35,119 [RapidOCR] download_file.py:60: File exists and is valid: /usr/loc

number of chunks 68
## How AI Impacts Skill Formation

Judy Hanwen Shen

∗

Alex Tamkin †

February 3, 2026

## Abstract

AI assistance produces significant productivity gains across professional domains, particularly for novice workers. Yet how this assistance affects the development of skills required to effectively supervise AI remains unclear. Novice workers who rely heavily on AI to complete unfamiliar tasks may compromise their own skill acquisition in the process. We conduct randomized experiments to study how developers gained mastery of a new asynchronous programming library with and without the assistance of AI. We find that AI use impairs conceptual understanding, code reading, and debugging abilities, without delivering significant efficiency gains on average. Participants who fully delegated coding tasks showed some productivity improvements, but at the cost of learning the library. We identify six distinct AI interaction patterns, three of which involve cognitive engageme

In [ ]:
print(chunks[2])

Software engineering, in particular, has been identified as a profession in which AI tools can be readily applied and AI assistance significantly improves productivity in daily tasks [Peng et al., 2023, Cui et al., 2024]. Junior or novice workers, in particicular, benefit most from AI assistance when writing code. In high-stakes applications, AI written code may be debugged and tested by humans before a piece of software is ready for deployment. This additional verification that enhances safety is only possible when human engineers themselves have the skills to understand code and identify errors. As AI development progresses, the problem of supervising more and more capable AI systems becomes more difficult if humans have weaker abilities

∗ Work done as a part of the Anthropic Fellows Program, judy@anthropic.com

† Anthropic, atamkin@anthropic.com

Figure 1: Overview of results: (Left) We find a significant decrease in library-specific skills (conceptual understanding, code reading, 

### Save document chunks to Chroma collection
Create a new collection to store all the document chunks


In [ ]:
collection = client.get_or_create_collection(
    name="AI_collection",
    embedding_function=embedding_func,
)

collection.add(
    documents=chunks,
    ids=[f"id_{i}" for i in range(len(chunks))]
)

print(f"Successfully create chroma vector database collection!")

Successfully create chroma vector database collection!


Now we have created a chroma vector database for your uploaded document! You can go to **Section 4** to redefine the RAG agent to connect to the new collection and start Q&A over your document!

### Parsing the figures in document (optional)


In [ ]:
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.document_converter import DocumentConverter
from docling.datamodel.base_models import InputFormat
from docling.document_converter import PdfFormatOption
import os
import shutil

# 1. Configure pipeline to capture images
pipeline_options = PdfPipelineOptions()
pipeline_options.generate_picture_images = True
pipeline_options.images_scale = 2.0  # Higher quality

# 2. Assign options specifically to the PDF format
converter = DocumentConverter(
    format_options={
        InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
    }
)

result = converter.convert(input_doc)

# 3. Create output folder. Delete the folder if existing
folder_name = "figures"
if os.path.exists(folder_name):
    print(f"Deleting existing folder: {folder_name}")
    shutil.rmtree(folder_name)
    print(f"Successfully deleted {folder_name}")

# 4. Create the new folder for output
else:
    os.makedirs(folder_name)
    print(f"Successfully created new folder: {folder_name}")

# 5. Use get_image() and Pillow to save
for i, element in enumerate(result.document.pictures):
    print("Extracting figure: ", i)
    # This retrieves the actual PIL image object
    pil_img = element.get_image(result.document)

    if pil_img:
        # Format i as a three-digit string
        pil_img.save(f"./{folder_name}/figure_{i:03d}.png", "PNG")

[INFO] 2026-02-05 22:55:05,704 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2026-02-05 22:55:05,725 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.onnx
[INFO] 2026-02-05 22:55:05,726 [RapidOCR] main.py:53: Using /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.onnx
[INFO] 2026-02-05 22:55:05,847 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2026-02-05 22:55:05,853 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_ppocr_mobile_v2.0_cls_infer.onnx
[INFO] 2026-02-05 22:55:05,853 [RapidOCR] main.py:53: Using /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_ppocr_mobile_v2.0_cls_infer.onnx
[INFO] 2026-02-05 22:55:05,910 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2026-02-05 22:55:05,946 [RapidOCR] download_file.py:60: File exists and is valid: /usr/loc

Successfully created new folder: figures
Extracting figure:  0
Extracting figure:  1
Extracting figure:  2
Extracting figure:  3
Extracting figure:  4
Extracting figure:  5
Extracting figure:  6
Extracting figure:  7
Extracting figure:  8
Extracting figure:  9
Extracting figure:  10
Extracting figure:  11
Extracting figure:  12
Extracting figure:  13
Extracting figure:  14
Extracting figure:  15
Extracting figure:  16
Extracting figure:  17
Extracting figure:  18
Extracting figure:  19
Extracting figure:  20
Extracting figure:  21
Extracting figure:  22
Extracting figure:  23
Extracting figure:  24
Extracting figure:  25
